In [4]:
import os 
import sys
from tqdm import tqdm
import importlib
import numpy as np
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt



import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Input, LSTM, Embedding, Reshape, Dense, Dropout, Conv1D, Conv2D, MaxPooling1D, Activation, Multiply, Add, Lambda, BatchNormalization
from tensorflow.keras import regularizers


module_path = "/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/"
data_path = "/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/data/long_seqs_v10/"

if module_path not in sys.path:
    sys.path.append(module_path)

In [11]:
BATCH_SIZE = 64
T = 20 + 1
VOCAB = ['END/PADDING', 'INIT', 'start', 'view', 'click', 'install']
EVENT_VOCAB_DIM = len(VOCAB)
LATENT_DIM = EMB_DIM = 32
HIDDEN_DIM = 100

END_TOKEN = 0

In [53]:
pos_data_path = os.path.join(data_path, 'positive_long_sequences.pickle')
#neg_data_path = os.path.join(data_path, 'negative_long_sequences.pickle')


def create_dataset(features: np.array, labels: np.array, batch_size=2, epochs=10, buffer_size=10000):
    """
    Create dataset from numpy arrays
    """
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.shuffle(buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    return dataset


def load_sequence_from_pickle_to_numpy(pickle_file_path):
    """
        A list of sequence in format of (event_type, time_delta)
    :param pickle_file_path: e.g. /.../project-basileus/seq-gan/data/fixed_length/valid_sequences.pickle
    :return: (event_type_seqs, time_delta)
    """
    with open(pickle_file_path, 'rb') as f:
        raw_seqs = pickle.load(f)

    if not raw_seqs or not raw_seqs[0]:
        return np.array([]), np.array([])

    N = len(raw_seqs)
    T = len(raw_seqs[0])
    
    seqs = np.array(raw_seqs)
#     print(seqs.shape)
    
    et_seqs = seqs[:, :, 0].astype(np.float64).reshape((N, T, 1))
    ts_seqs = seqs[:, :, 1].astype(np.float64).reshape((N, T, 1))
    return et_seqs, ts_seqs
    
pos_event_type_seqs, pos_timestamp_seqs = load_sequence_from_pickle_to_numpy(pos_data_path)
#neg_event_type_seqs, neg_timestamp_seqs = load_sequence_from_pickle_to_numpy(neg_data_path)

In [13]:
z = np.float32(np.random.normal(size=[BATCH_SIZE, T, LATENT_DIM]))

In [41]:
def build_G(batch_size, seq_length, latent_dim, hidden_dim=11):
    # normal LSTM
    i_noise = Input(batch_shape=(batch_size, seq_length, latent_dim), name='noise')  

    whole_seq_output, final_memory_state, final_carry_state = LSTM(hidden_dim, 
                       name='lstm',
                       stateful=True,
                       return_sequences=True,
                       return_state = True, #from RC-GAN
                       unit_forget_bias=True, #from RC-GAN
                       kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
                       recurrent_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
                       bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(i_noise)

    ts = Dense(1,
                   activation='relu',
                   name='dense_1',
                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
                   bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(whole_seq_output)    

    generator = Model(
        inputs=i_noise,
        outputs=ts)
    
    return generator

In [44]:
G = build_G(batch_size=BATCH_SIZE, seq_length=T, latent_dim=LATENT_DIM, hidden_dim=HIDDEN_DIM)

In [45]:
G(z)

<tf.Tensor: shape=(64, 21, 1), dtype=float32, numpy=
array([[[6.0379833e-02],
        [1.2095232e-01],
        [0.0000000e+00],
        ...,
        [9.5117249e-02],
        [1.9538330e-01],
        [2.8514573e-02]],

       [[3.4092762e-02],
        [0.0000000e+00],
        [0.0000000e+00],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [2.2611817e-02]],

       [[0.0000000e+00],
        [0.0000000e+00],
        [3.4844577e-02],
        ...,
        [2.1984738e-01],
        [1.3332030e-02],
        [1.2777551e-01]],

       ...,

       [[6.0366184e-02],
        [2.3596005e-01],
        [2.2036317e-01],
        ...,
        [1.0991925e-01],
        [3.4039028e-02],
        [0.0000000e+00]],

       [[0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00]],

       [[9.5774598e-02],
        [2.3612042e-01],
        [1.3928674e-04],
        ...,
        [0.0000000e+0

In [79]:
def build_D(T, latent_dim, hidden_dim=11):
    i_ts = Input(shape=(T, 1), name='time_stamps')
    
    whole_seq_output, final_memory_state, final_carry_state = LSTM(hidden_dim, 
                       name='lstm',
                       #stateful=True,
                       return_sequences=False,
                       return_state = True, #from RC-GAN
                       unit_forget_bias=True, #from RC-GAN
                       kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
                       recurrent_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
                       bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(i_ts)
    

    logits = Dense(1,
                   activation='linear',
                   name='dense_1',
                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
                   bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(whole_seq_output)    
    
    prob = tf.math.sigmoid(logits)
    
    discriminator = Model(
        inputs=i_ts,
        outputs=[prob, logits])

    return discriminator

In [80]:
D = build_D(T=T, latent_dim=LATENT_DIM, hidden_dim=HIDDEN_DIM)

In [81]:
D(G(z))

[<tf.Tensor: shape=(64, 1), dtype=float32, numpy=
 array([[0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
        [0.49586233],
    

## RC-GAN Traning

In [82]:
def train_generator(generator, discriminator, batch_size, T, latent_dim, mc_size=10, verbose=False,                   
                    optimizer=Adam(lr=0.001), end_token=0, init_token=1):
            
    z = np.float32(np.random.normal(size=[batch_size, T, latent_dim]))
    
    with tf.GradientTape() as tape:                        

        G_sample = generator(z)
        D_fake, D_logit_fake = discriminator(G_sample)

        generator_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))
        
    if verbose:
        print('generator loss:{}'.format(generator_loss))
        print('-----------------------')

    # update generator
    generator_grads = tape.gradient(generator_loss, generator.trainable_variables)
    optimizer.apply_gradients(zip(generator_grads, generator.trainable_variables))

    return generator_loss


def train_discriminator(real_data_batch, generator, discriminator, batch_size, T, latent_dim, verbose=False,
                        optimizer=Adam(lr=0.001)):

    # data prep
    real_labels = tf.ones((batch_size, 1))        
    fake_labels = tf.zeros((batch_size, 1))    
    
    z = np.float32(np.random.normal(size=[batch_size, T, latent_dim]))
    fake_data_batch = np.array(generator(z))
    
    total_data = tf.concat([fake_data_batch, real_data_batch], axis=0)
    total_labels = tf.concat([fake_labels, real_labels], axis=0)
        
    # train the discriminator
    with tf.GradientTape() as tape:                                                           
        # train discriminator
        pred_prob, pred_logits = discriminator(total_data)

        # cross-entropy loss
        discriminator_loss = tf.reduce_mean(
            tf.keras.losses.binary_crossentropy(total_labels, pred_prob, from_logits=False))

        if verbose:
            print('total discriminator loss:{}'.format(discriminator_loss))
            print('-----------------------')

    grads = tape.gradient(discriminator_loss, discriminator.trainable_variables)
    optimizer.apply_gradients(zip(grads, discriminator.trainable_variables))

    return discriminator_loss

In [89]:
G = build_G(batch_size=BATCH_SIZE, seq_length=T, latent_dim=LATENT_DIM, hidden_dim=HIDDEN_DIM)

D = build_D(T=T, latent_dim=LATENT_DIM, hidden_dim=HIDDEN_DIM)

In [90]:
N_total_G = pos_timestamp_seqs.shape[0]

EPOCHS = 3
_TOTAL_STEPS = int(EPOCHS * N_total_G / BATCH_SIZE)

train_dataset = create_dataset(pos_timestamp_seqs[:N_total_G,:,:],
                             np.ones((N_total_G, 1)),
                             batch_size=BATCH_SIZE,
                             epochs=EPOCHS,
                             buffer_size=N_total_G)

gen_loss_history = []
disc_loss_history = []

step = 0

G_optimizer = SGD(learning_rate=1e-3)
D_optimizer = SGD(learning_rate=1e-4)

_G_STEPS = 2
_D_STEPS = 1

In [91]:
for time_stamp_batch, _ in tqdm(train_dataset.take(_TOTAL_STEPS)):
    step += 1
    print('Training Step:', step)
    # train the generator
    for _ in range(_G_STEPS):
        gen_loss = train_generator(G, D, BATCH_SIZE, T, LATENT_DIM, mc_size=100, verbose=True, optimizer=G_optimizer)
        gen_loss_history.append(gen_loss.numpy())
    
    # train the discriminator
    for _ in range(_D_STEPS):
        disc_loss = train_discriminator(time_stamp_batch, G, D, BATCH_SIZE, T, LATENT_DIM, verbose=True, optimizer=D_optimizer)
        disc_loss_history.append(disc_loss.numpy())

0it [00:00, ?it/s]

Training Step: 1
generator loss:0.5752772092819214
-----------------------
generator loss:0.5757743120193481
-----------------------

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



1it [00:00,  2.10it/s]

total discriminator loss:0.8761799335479736
-----------------------
Training Step: 2
generator loss:0.5755796432495117
-----------------------
generator loss:0.5759454369544983
-----------------------


2it [00:00,  2.76it/s]

...

total discriminator loss:0.6970522403717041
-----------------------
Training Step: 7226
generator loss:0.8400783538818359
-----------------------
generator loss:0.8408255577087402
-----------------------


7226it [34:32,  3.44it/s]

total discriminator loss:0.7035697102546692
-----------------------
Training Step: 7227
generator loss:0.7964147329330444
-----------------------
generator loss:0.7970933318138123
-----------------------


7227it [34:32,  3.49it/s]

total discriminator loss:0.7069874405860901
-----------------------
